# Kotlin Coroutines: SupervisorJob and supervisorScope

This notebook explains what SupervisorJob and supervisorScope are, when to use them, and how they differ from regular Job and coroutineScope. Each section includes small runnable examples.

Key idea: In a supervisor, one child’s failure does not cancel its siblings. In regular structured concurrency, a child failure cancels the whole family.


In [3]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")

## 1) The problem: child failure cancels siblings by default
With a regular Job (or inside coroutineScope), if one child fails, the parent gets canceled and all other children are canceled too.

The example below shows that Task B never finishes because Task A throws.

In [4]:
import kotlinx.coroutines.*

runBlocking {
    try {
        coroutineScope { // regular, non-supervised
            launch {
                delay(100)
                println("[regular] A: about to fail")
                throw RuntimeException("Boom from A")
            }
            launch {
                try {
                    delay(300)
                    println("[regular] B: finished (won't happen)")
                } finally {
                    println("[regular] B: finally (canceled)")
                }
            }
        }
    } catch (e: Throwable) {
        println("[regular] Parent caught: ${e::class.simpleName} -> ${e.message}")
    }
    println("[regular] After coroutineScope")
}


[regular] A: about to fail
[regular] B: finally (canceled)
[regular] Parent caught: RuntimeException -> Boom from A
[regular] After coroutineScope


## 2) supervisorScope: isolate children from each other
supervisorScope creates a scope where failure of one child does not cancel its siblings. The scope still completes exceptionally with the first failure (unless you handle it), but it waits for all children to finish first.

Below, A fails; B still completes; the parent then observes the exception after both are done.

In [5]:
import kotlinx.coroutines.*

runBlocking {
    try {
        supervisorScope {
            launch {
                delay(100)
                println("[supervisorScope] A: about to fail")
                throw RuntimeException("Boom from A")
            }
            launch {
                try {
                    delay(300)
                    println("[supervisorScope] B: finished (still runs)")
                } finally {
                    println("[supervisorScope] B: finally")
                }
            }
        }
    } catch (e: Throwable) {
        println("[supervisorScope] Parent caught: ${e::class.simpleName} -> ${e.message}")
    }
    println("[supervisorScope] After supervisorScope")
}


[supervisorScope] A: about to fail
[supervisorScope] B: finished (still runs)
[supervisorScope] B: finally
[supervisorScope] After supervisorScope


## 3) SupervisorJob: a Job with supervisor semantics for a CoroutineScope
SupervisorJob is a Job implementation you put into a CoroutineScope to get supervisor semantics for all children launched in that scope. A child failure:
- does NOT cancel the scope
- does NOT cancel its siblings
- is reported to a CoroutineExceptionHandler (if uncaught)

You typically combine it with a handler to observe failures.

In [6]:
import kotlinx.coroutines.*

val handler = CoroutineExceptionHandler { _, e ->
    println("[SupervisorJob] Caught by handler: ${e::class.simpleName} -> ${e.message}")
}

val supervisedScope = CoroutineScope(SupervisorJob() + Dispatchers.Default + handler)

runBlocking {
    val a = supervisedScope.launch {
        delay(100)
        println("[SupervisorJob] A: about to fail")
        throw RuntimeException("Boom from A")
    }
    val b = supervisedScope.launch {
        try {
            delay(300)
            println("[SupervisorJob] B: finished (not canceled)")
        } finally {
            println("[SupervisorJob] B: finally")
        }
    }

    // Wait for both children
    joinAll(a, b)
    println("[SupervisorJob] Both children joined; scope is still active = ${supervisedScope.isActive}")
}


[SupervisorJob] A: about to fail
[SupervisorJob] Caught by handler: RuntimeException -> Boom from A
[SupervisorJob] B: finished (not canceled)
[SupervisorJob] B: finally
[SupervisorJob] Both children joined; scope is still active = true


## 4) Cancellation still flows downward
Even with supervisor semantics, canceling the parent scope cancels all its children.

In [7]:
import kotlinx.coroutines.*

val cancelScope = CoroutineScope(SupervisorJob())

runBlocking {
    val job = cancelScope.launch {
        launch { try { delay(1_000); println("[cancel] Child 1 done") } finally { println("[cancel] Child 1 finally") } }
        launch { try { delay(1_000); println("[cancel] Child 2 done") } finally { println("[cancel] Child 2 finally") } }
    }
    delay(200)
    println("[cancel] Cancel parent scope")
    cancelScope.cancel()
    job.join()
    println("[cancel] Parent canceled; children are canceled too")
}


[cancel] Cancel parent scope
[cancel] Child 1 finally
[cancel] Child 2 finally
[cancel] Parent canceled; children are canceled too


## 5) Quick guidelines
- Use supervisorScope for a small region where siblings must be isolated from each other, but where the region should still fail overall if any child fails (after waiting for others).
- Use a CoroutineScope(SupervisorJob()) when you need a long-lived scope (e.g., UI ViewModel, service) where one child’s failure should not bring down its siblings or the scope.
- Always attach a CoroutineExceptionHandler when using SupervisorJob if you don’t handle exceptions in children explicitly, to avoid silent failures.